```
File: linked_in_api.ipynb
Author: Michael Lucky
Date: September 13, 2023
Description: This notebook is intended to be used as a template for using the LinkedIn API to extract data from LinkedIn company profiles for data collection.

Copyright (c) 2023 Jelloow

For inquiries or permissions regarding the use of this code, please contact:
info@jelloow.com
```

In [3]:
import http.client
import logging
import json
import datetime
import os
from dotenv import load_dotenv

# format logging
logging.basicConfig(format='%(asctime)s: %(name)s: %(levelname)s: %(message)s', level=logging.DEBUG, datefmt='%Y-%m-%dT%H:%M:%S%z')

# create logger
log = logging.getLogger(__name__)

# read in .env file
load_dotenv()

LINKEDIN_CLIENT_SECRET = os.environ['LINKEDIN_CLIENT_SECRET']
LINKEDIN_CLIENT_ID = os.environ['LINKEDIN_CLIENT_ID']
LINKEDIN_ACCESS_TOKEN = os.environ['LINKEDIN_ACCESS_TOKEN']
LINKEDIN_ACCESS_TOKEN_EXPIRATION = os.environ['LINKEDIN_ACCESS_TOKEN_EXPIRATION']

In [4]:
# create a connection to linkedin
LINKEDIN_BASE_URL = "api.linkedin.com"
LINKEDIN_VERSION = "202308"
conn = http.client.HTTPSConnection(LINKEDIN_BASE_URL)

In [6]:
# Get the access token

payload = f"grant_type=client_credentials&client_id={LINKEDIN_CLIENT_ID}&client_secret={LINKEDIN_CLIENT_SECRET}"
headers = {
    'content-type': "application/x-www-form-urlencoded"
    }
conn.request("POST", "/oauth/v2/accessToken", payload, headers)
res = conn.getresponse()
if res.status >= 200 and res.status < 300:

    # create a timestamp
    timestamp = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S%z')

    # read the response
    data = res.read()

    # parse the response
    data = json.loads(data.decode('utf-8'))
    LINKEDIN_ACCESS_TOKEN = data['access_token']
    expires_in = data['expires_in']

    # add the expires in time to the timestamp
    expires_in = datetime.timedelta(seconds=expires_in)

    # save the access token to the .env file
    env = {}
    env['LINKED_IN_ACCESS_TOKEN'] = LINKEDIN_ACCESS_TOKEN
    env['LINKED_IN_ACCESS_TOKEN_EXPIRATION'] = expires_in

    # write the .env file
    with open('./.env', 'w') as f:
        f.writelines([f'{key}="{value}"' for key, value in env.items()])
else:
    log.error(f"Error: {res.read()}")
    log.error(f"Error: {res.status} {res.reason}")

conn.close()

2023-09-22T11:18:47-0600: __main__: ERROR: Error: b'{"error":"access_denied","error_description":"This application is not allowed to create application tokens"}'
2023-09-22T11:18:47-0600: __main__: ERROR: Error: 401 Unauthorized


In [5]:
payload = ''
headers = {
    'Authorization': f'Bearer {LINKEDIN_ACCESS_TOKEN}',
    'LinkedIn-Version': f'{LINKEDIN_VERSION}',
    'X-Restli-Protocol-Version': '2.0.0'
}

domain = "gmail.com"
conn.request(
    "GET", f"/rest/organizations?q=emailDomain&emailDomain={domain}",
    payload,
    headers
)

res = conn.getresponse()
if res.status >= 200 and res.status < 300:
    data = res.read()
    log.debug(data.decode("utf-8"))
else:
    log.error(f"Error: {res.read()}")
    log.error(f"Error: {res.status} {res.reason}")

conn.close()


2023-09-22T11:18:40-0600: __main__: ERROR: Error: b'{"status":403,"serviceErrorCode":100,"code":"ACCESS_DENIED","message":"Not enough permissions to access: organizations.FINDER-emailDomain.20230801"}'
2023-09-22T11:18:40-0600: __main__: ERROR: Error: 403 Forbidden
